##This notebook is an exampe of a telegaram bot minimal user interface.

In [ ]:
pip install telebot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213956 sha256=2c7590dedf4c43a4dace7811efcedf9f503164abe4f17afb6ffc9202ab8bf7e7
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pyTelegramBotAPI


In [ ]:
import telebot
import psycopg2
from telebot import types

# Instantiate the bot
my_bot = telebot.TeleBot('')

# Global variables to hold user details
user_name = ''
user_surname = ''
user_age = 0
user_id = 0
user_car_plate = ''

# Bot command to initiate the registration process
@my_bot.message_handler(content_types=['text'])
def start(message):
    if message.text == '/reg':
        my_bot.send_message(message.from_user.id, "What is your name?")
        my_bot.register_next_step_handler(message, retrieve_name)
    else:
        my_bot.send_message(message.from_user.id, 'Please, write /reg command to register.')

# Function to get user's name
def retrieve_name(message):
    global user_name
    global user_id
    user_id = message.from_user.id
    user_name = message.text
    my_bot.send_message(message.from_user.id, 'What is your surname?')
    my_bot.register_next_step_handler(message, retrieve_surname)

# Function to get user's surname
def retrieve_surname(message):
    global user_surname
    user_surname = message.text
    my_bot.send_message(message.from_user.id,'What is your car plate number??!!')
    my_bot.register_next_step_handler(message, retrieve_car_plate)

# Function to get user's car plate
def retrieve_car_plate(message):
    global user_car_plate
    user_car_plate = message.text
    my_bot.send_message(message.from_user.id,'How old are you?')
    my_bot.register_next_step_handler(message, retrieve_age)

# Function to get user's age
def retrieve_age(message):
    global user_age
    user_age = int(message.text)
    confirmation_keyboard = types.InlineKeyboardMarkup()
    key_yes = types.InlineKeyboardButton(text='Yes', callback_data='yes')
    confirmation_keyboard.add(key_yes)
    key_no = types.InlineKeyboardButton(text='No', callback_data='no')
    confirmation_keyboard.add(key_no)
    confirmation_text = f'You are {user_age} years old, your full name is {user_name} {user_surname} and your car plate number is: {user_car_plate}. Is this information correct?'
    my_bot.send_message(message.from_user.id, text=confirmation_text, reply_markup=confirmation_keyboard)

# Function to insert user's data into database
def insert_user(tg_id, user_name, user_surname, car_plate, user_age):
    conn = psycopg2.connect("host=109.120.191.148 dbname=PostgreSQL-4323 user=user password="")
    cursor = conn.cursor()
    insert_query = """
        INSERT INTO users (tg_id, user_name, user_surname, car_plate, user_age)
        VALUES (%s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, (tg_id, user_name, user_surname, car_plate, user_age))
    conn.commit()
    cursor.close()
    conn.close()

# Bot callback function for confirmation response
@my_bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    if call.data == "yes":
        insert_user(user_id, user_name, user_surname, user_car_plate, user_age)
        my_bot.send_message(call.message.chat.id, 'You are now registered : )');
    elif call.data == "no":
        my_bot.send